This notebook is my take on the Kaggle competition on Natural Language Processing (NLP) to predict which Tweets are about real disasters and which one’s aren’t. I'm joining this competition while following a Coursera course on Deep Learning, so I am restricted to use a Recurrent Neural Network as classifier.

First I will explore the available data in order to be able to form a master plan for achieving a high score in the competition.

In [12]:
import pandas as pd

df_train = pd.read_csv('input/train.csv')
df_train.sample(10)


,id,keyword,location,text,target
5750,8210,riot,"Detroit, MI",To All The Meat-Loving Feminists Of The World ...,0
7095,10164,violent%20storm,"Watertown, Mass.",Violent Storm Causes Damage Flooding in Watert...,1
1330,1921,burning%20buildings,"Sacramento, CA",Killing Black Babies at Planned Parenthood whe...,1
6424,9185,suicide%20bomber,nigeria,Suicide Bomber Kills More Than a Dozen in Saud...,1
2182,3126,debris,NaN,#??? #?? #??? #??? MH370: Aircraft debris foun...,1
6283,8974,storm,pittsboro,Ready for this storm,1
5385,7685,panic,NaN,#dream #magic The #linden method lite version ...,0
5536,7897,radiation%20emergency,NaN,I liked a @YouTube video http://t.co/V57NUgmGK...,1
4726,6722,lava,Jakarta,@YoungHeroesID Lava Blast &amp; Power Red @die...,0
678,979,blazing,"Dallas, TX",Bright &amp; BLAZING Fireman Birthday Party ht...,0


In [10]:
df_test = pd.read_csv('input/test.csv')
df_test.sample(10)

,id,keyword,location,text
778,2567,crash,"Austin, TX",lol at the guy whipping by me on a double yell...
94,319,annihilated,NaN,@jacksfilms #yiayplan well first we strike dre...
165,545,army,Mexico! ^_^,5 Seconds of Summer Is my pick for http://t.co...
632,2062,casualty,Canada | #LUX,@Rx_Casualty @Vpzedd @Daintt_M call
2220,7411,obliterated,MIA,Shout out to fishtails your .75 cent natty lig...
2835,9412,survivors,Orlando,#orlando Survivors of Shanghai Ghetto reunite ...
1687,5701,floods,"Guayaquil, Ecuador",I'm so dried out in this desert let me know I ...
2464,8229,riot,NYC,Matt Baume Digs Into the the Controversial Û÷...
1927,6498,injuries,North Carolina,@RVacchianoNYDN @NYDNSports glad to see a care...
1327,4380,earthquake,NaN,5.3 #Earthquake in South Of Fiji Islands. #iPh...


Great, the training data contains 5 columns, id, keyword, location, text and target. The test file (needed for the final submission only misses the target column, which is as expected. Let's see which columns we can use by exploring them.

In [13]:
df_train.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [14]:
df_test.isna().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [15]:
df_train.shape

(7613, 5)

In [16]:
df_test.shape

(3263, 4)

So, roughly 1/3 of the entries do not contain a location, which makes this column not suitable as input for a classifier. The keyword is not missing that often, but we have to see if the keyword is meaningful enough to keep as a feature (and drop the rows with missing keywords).

In [23]:
print('Total number of entries={}, Unique ids={}, unique keywords={}, unique locations={}'.format(len(df_train),len(df_train.id.unique()),len(df_train.keyword.unique()),len(df_train.location.unique())))

Total number of entries=7613, Unique ids=7613, unique keywords=222, unique locations=3342


In [34]:
df_train[['keyword','target']].groupby('keyword').value_counts()

keyword     target
ablaze      0         23
            1         13
accident    1         24
            0         11
aftershock  0         34
                      ..
wreck       0         30
            1          7
wreckage    1         39
wrecked     0         36
            1          3
Length: 438, dtype: int64